In [140]:
import numpy as np
import torch
import math
import importlib
import socialSigLayers
importlib.reload(socialSigLayers)
import pandas as pd
import random


###### Define our model
class SocialSigNet(torch.nn.Module):
    def __init__(self, X):
        super().__init__()
        self.SocialSig = socialSigLayers.bilinearImputation(X=X)                
        self.maxPool = torch.nn.MaxPool2d(kernel_size=(10,10))  #10,10 is static here.  Will need to be dynamic based on user dim settings.

    def forward(self, X):
        out = self.SocialSig(X)
        out = self.maxPool(out)
        
        return out

####### Load our Data
devSet = pd.read_csv("us_migration.csv")
devSet = devSet.loc[:, ~devSet.columns.str.contains('^Unnamed')]
devSet = devSet.apply(lambda x: pd.to_numeric(x, errors='coerce'))
devSet = devSet.dropna(axis=1)

#y - 'number_moved'
#x - 'everything else that is or can be represented as a float.'
y = torch.Tensor(devSet['US_MIG_05_10'].values)
X = devSet.loc[:, devSet.columns != "US_MIG_05_10"].values
####### Build and fit the Model
model = SocialSigNet(X=X)
lr = 1
batchSize = 8

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr = lr)


def update_function(param, grad, loss, learning_rate):
    return param - learning_rate * grad


for t in range(2):
    #Batches
    batchObs = random.sample(range(0, len(y)),batchSize)
    modelX = X[batchObs]
    modely = y[batchObs]
    
    print("EPOCH: ", t)
    y_pred = model(modelX)
    
    #print("    Predicted Y:    ", y_pred)
    loss = criterion(y_pred, modely)
    print("    Loss:           ", loss)

    # Zero gradients, perform a backward pass, and update the weights.
    #optimizer.zero_grad()
    print(model(X))
    grad = torch.autograd.grad(outputs=loss, inputs=modelX, retain_graph = True)
    #print("    Gradient:       ", grad[0])
    loss.backward()
    optimizer.step()
    # https://discuss.pytorch.org/t/updatation-of-parameters-without-using-optimizer-step/34244/4
    with torch.no_grad():
        for p in model.parameters():
            print("    In with:        ", p.data)
            new_val = update_function(p, grad[0], loss, lr)
            p.copy_(new_val)
    
    print("\n")

EPOCH:  0


AttributeError: module 'torch' has no attribute 'Variable'